#### LangChain 필요성

- OpenAI API 직접 사용시 
    - 사용자 질문 -> LLM 호출 -> 응답

- 실제 서비스에서 필요한 기능
    - PDF 문서 안에서 답 찾기
    - DB 조회 후 결과 설명
    - 이전 대화 기억
    - 여러 단계 추론

#### LangChain 핵심 개념
- Chain : 여러 단계를 연결한 처리 흐름
    - ex) 검색 + 요약 + 생성 같은 다단계 흐름 구성 가능
- Prompt Template : 프롬프트를 템플릿화
- Memory : 이전 대화 저장
- Retriever (검색기)
- Agent : LLM 이 스스로 도구를 선택하여 실행

#### 활용 가능 분야
- 문서기반 QA
    - ex) 사내문서 QA 챗봇, PDF 기반 질의응답, 법률 문서 검색, 기술 매뉴얼 검색
- 챗봇 시스템
- 데이터 분석 AI
- 에이전트 기반 자동화 시스템

### RAG(Retrieval Augmented Generation)
- 검색 증강 생성
- 특정 자료(문서, PDF, DB 등)에 대해 질문에 답할 수 있다.
- 2가지 방식 제공
    - RAG Agent 방식 : Agent 를 이용해 여러번 검색이 가능하고 복잡한 질문 처리 가능
    - Two-step RAG Chain : 단순한 형태 / 한 번 호출

- 처리 단계
    - 1) Indexing : 데이터를 어떤 소스로부터 가져와서 검색할 수 있도록 정리(PDF->텍스트 추출->분할->임베딩->벡터DB 저장장)
    - 2) Retrieval and Generation : 실제 RAG가 동작하는 단계(사용자가 질문을 입력하면 인덱스에서 관련 데이터를 검색하고 그 데이터를 모델에 전달하여 답변을 생성)

#### 정리
- LLM 기반 기능을 체계적으로 사용할 수 있도록 제공되는 오픈소스 프레임워크
- 외부 데이터 연결, 문서검색, 도구사용(API 호출), 멀티스텝 추론, 메모리 유지 대화 같은 복합적인 AI 애플리케이션 구조를 쉽게 만들 수 있도록 도와주는 도구

<img src="https://xeblog.s3.ap-northeast-2.amazonaws.com/langchain_builder_9f58224358.jpg" width="700" height="604">

In [40]:
from dotenv import load_dotenv, find_dotenv
import bs4
from pathlib import Path

load_dotenv(find_dotenv())

True

In [2]:
!pip install langchain_openai

   ---------------------------------------- 0.0/921.1 kB ? eta -:--:--
   ---------------------------------------- 921.1/921.1 kB 27.8 MB/s  0:00:00

   ---------------------------------------- 0/3 [regex]
   -------------------------- ------------- 2/3 [langchain_openai]
   -------------------------- ------------- 2/3 [langchain_openai]
   ---------------------------------------- 3/3 [langchain_openai]




[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# 랭체인에서 제공하는 로더 
from langchain_community.document_loaders import YoutubeLoader, WebBaseLoader, PyPDFLoader 

# 임베딩 처리
from langchain_community.vectorstores import FAISS

# 문서 내용 임베딩
from langchain_openai import OpenAIEmbeddings
# 모델 생성
from langchain_openai import ChatOpenAI
from langchain.chat_models import init_chat_model

# 메세지
from langchain.messages import HumanMessage, AIMessage, SystemMessage

# 프롬프트 작성
from langchain_core.prompts import PromptTemplate 

# 글을 쪼갤 때 사용하는 라이브러리
from langchain_text_splitters  import RecursiveCharacterTextSplitter

# Tools
from langchain.tools import tool
from langchain.agents import create_agent

# 크롤링
from langchain_community.document_loaders.firecrawl import FireCrawlLoader

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser

c:\source\pythonsource\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:25: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
# 모델 생성
# 방법 1
# model = init_chat_model(model="gpt-4.1-mini", temperature=0,max_tkes=500,timeout=60)
# 방법2
model = ChatOpenAI(model="gpt-4.1-mini")



##### Langchain
- 모델 호출
    - 1) invoke() : 모델이 답변을 전체 생성 후 응답
    - 2) stream() : 모델이 생성한 답변을 중간중간 응답(논문요약, 보고서 생성, 코드 생성, 긴 설명)
    - 3) batch() : 여러 개 요청을 한 번에 처리(병렬 처리 가능 - 리뷰 100 개 요약, 문단 50개 번역..)

In [6]:
# 모델 호출 : invoke()
# 하나의 메시지 입력 시 
res = model.invoke("사과의 효능을 설명해줘")
print(res)
print(res.usage_metadata)
print(res.content)

content='사과는 영양가가 풍부한 과일로 여러 건강상의 이점을 제공합니다. 주요 효능은 다음과 같습니다:\n\n1. **풍부한 항산화제 함유**  \n   사과에는 비타민 C, 플라보노이드, 폴리페놀 등 항산화제가 많이 들어 있어 체내 활성산소를 제거하고 세포 손상을 예방하는 데 도움을 줍니다.\n\n2. **소화기 건강 개선**  \n   사과에 함유된 식이섬유(특히 펙틴)는 장내 유익균의 성장을 촉진하고 배변을 원활하게 하여 소화기 건강을 돕습니다.\n\n3. **심혈관 질환 위험 감소**  \n   식이섬유와 항산화 물질이 혈중 콜레스테롤 수치를 낮추고 혈압을 조절해 심장병과 뇌졸중 예방에 도움을 줄 수 있습니다.\n\n4. **체중 관리**  \n   낮은 칼로리와 높은 섬유질 덕분에 포만감을 유지시켜 과식을 방지하고 체중 조절에 유리합니다.\n\n5. **혈당 조절**  \n   사과 속의 천천히 소화되는 섬유소가 혈당의 급격한 상승을 막아 당뇨병 관리에 도움이 됩니다.\n\n6. **뼈 건강 증진**  \n   사과에 들어있는 비타민 C와 다양한 미네랄이 뼈의 밀도를 유지하고 골밀도 감소를 예방하는 데 기여할 수 있습니다.\n\n이 외에도 사과는 피로 회복, 면역력 강화 등에 긍정적인 영향을 주므로 꾸준히 섭취하는 것이 좋습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 383, 'prompt_tokens': 16, 'total_tokens': 399, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'mo

In [7]:
# stream() : 모델이 생성한 답변을 중간중간 응답
agent = create_agent(model)

# stream_mede = "values" : 현재 상태 전체
# stream_mede = "update" : update 된 내용
for step in agent.stream({"messages":[{"role":"user","content":"사과의 효능 설명"}]}, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

사과의 효능 설명
================================== Ai Message ==================================

사과는 맛도 좋고 건강에도 여러 가지 이점이 있는 과일입니다. 사과의 주요 효능을 간단히 설명드리면 다음과 같습니다:

1. **풍부한 식이섬유**  
   사과에는 펙틴이라는 수용성 식이섬유가 많이 들어 있어 장 건강에 도움을 주고 변비 예방에 효과적입니다. 또한 혈중 콜레스테롤 수치를 낮추는 데에도 도움을 줄 수 있습니다.

2. **항산화 작용**  
   사과에는 비타민 C, 퀘르세틴(quercetin), 폴리페놀 같은 항산화 물질이 포함되어 있어 체내 활성산소를 제거하고 세포 손상을 줄여줍니다. 이로 인해 노화 방지와 면역력 강화에 도움이 됩니다.

3. **심혈관 건강 개선**  
   사과의 식이섬유와 항산화 성분은 혈압을 낮추고 혈관을 보호하여 심장 질환 위험을 줄이는 데 기여합니다.

4. **혈당 조절**  
   사과 속의 식이섬유는 혈당의 급격한 상승을 막아 당뇨병 관리에 도움을 줄 수 있습니다.

5. **체중 관리**  
   낮은 칼로리에 포만감을 주는 사과는 다이어트 중간 간식으로 적합하며, 과식을 방지하는 데 도움을 줍니다.

6. **소화 촉진 및 해독 작용**  
   사과는 소화를 돕고 간 기능을 지원하여 체내 독소 배출에 긍정적인 영향을 줄 수 있습니다.

이처럼 사과는 다양한 면에서 건강에 유익한 과일로, 꾸준히 섭취하면 전반적인 건강 증진에 큰 도움을 받을 수 있습니다.


In [8]:
# batch()
inputs = [
    "사과의 효능 설명",
    "바나나의 효능 설명",
    "오렌지의 효능 설명",
]
res = model.batch(inputs)
for r in res:
    print(r.content)

사과는 맛이 좋고 영양가가 풍부한 과일로 여러 가지 건강상 이점을 제공합니다. 사과의 주요 효능은 다음과 같습니다:

1. **풍부한 식이섬유**  
   사과에는 식이섬유가 많이 들어 있어 소화를 돕고 변비를 예방하며 장 건강을 개선하는 데 도움을 줍니다.

2. **심장 건강 증진**  
   사과에 포함된 폴리페놀과 플라보노이드 등의 항산화 물질은 혈압을 낮추고 나쁜 콜레스테롤(LDL)을 줄여 심혈관 질환 위험을 감소시킵니다.

3. **체중 조절 도움**  
   칼로리가 낮고 포만감을 주는 식이섬유가 많아 다이어트 시 간식으로 좋고 과식을 방지하는 데 도움을 줍니다.

4. **혈당 조절**  
   사과에 함유된 식이섬유가 혈당 상승을 완만하게 하여 당뇨 관리에 긍정적인 영향을 미칩니다.

5. **항산화 효과**  
   비타민 C와 다양한 식물 화학물질이 활성산소를 제거해 세포 손상을 줄이고 노화 방지에 도움을 줍니다.

6. **뇌 건강**  
   일부 연구에 따르면 사과의 항산화 성분이 뇌 기능 향상과 알츠하이머병 같은 신경 퇴행성 질환 예방에 도움이 될 수 있습니다.

7. **면역력 강화**  
   비타민과 미네랄이 풍부해 신체 면역 기능을 강화하는 데 기여합니다.

이처럼 사과는 일상 식단에 쉽게 포함시킬 수 있으며, 건강 유지와 질병 예방에 다양한 도움을 줄 수 있는 과일입니다.
바나나는 영양가가 풍부하고 다양한 건강 효능이 있는 과일입니다. 주요 효능은 다음과 같습니다:

1. **에너지 공급**  
   바나나는 탄수화물이 풍부해 빠른 에너지 공급원으로 좋습니다. 운동 전후 간식으로 적합합니다.

2. **소화 개선**  
   바나나에는 식이섬유가 많이 들어 있어 소화를 돕고 변비 예방에 효과적입니다.

3. **심장 건강**  
   칼륨 함량이 높아 혈압을 조절하고 심장 질환 위험을 줄이는 데 도움을 줍니다.

4. **기분 개선 및 스트레스 완화**  
   바나나에는 세로토닌 분비를 촉진하는 트립토판이 포함되어 있어 기

- Messages
    - https://docs.langchain.com/oss/python/langchain/messages

In [9]:
system_msg = SystemMessage("You're a helpful assistant.")
human_msg = HumanMessage("Hello, how are you?")

res = model.invoke([system_msg,human_msg])
print(res.content)

Hello! I'm doing well, thank you. How can I assist you today?


In [10]:
# 여러 개의 메시지 리스트
# https://docs.langchain.com/oss/python/langchain/models#invoke
# 대화기록

message =[
        {
            "role" : "user",
            "content" : "2002 월드컵에서 가장 화제가 된 나라가 어딜까?"
        },
        {
            "role" : "assistant",
            "content" : "대한민국이 화제가 됨"
        },
        {
            "role" : "user",
            "content" : "그나라가 화제가 된 이유를 상세하게 설명해"
        },
    ]

res = model.invoke(message)
print(res.content)

2002년 FIFA 월드컵에서 가장 화제가 된 나라는 단연 대한민국입니다. 그 이유는 다음과 같습니다:

1. **역대 최고 성적 달성**  
   대한민국은 2002년 월드컵에서 사상 최초로 4강에 진출하는 쾌거를 이루었습니다. 이는 아시아 국가 중 최초로 이루어진 일이며, 월드컵 역사상 비유럽·남미권 국가가 4강에 오른 보기 드문 사례였습니다.

2. **공식 개최국 및 공동 개최국의 역할**  
   2002년 월드컵은 대한민국과 일본이 공동 개최한 첫 번째 월드컵이었고, 대한민국은 이 대회를 통해 축구 강국들과의 맞대결에서 강한 모습을 보여주었습니다. 특히 한일 월드컵 개최국이어서 국민적 관심과 열기가 매우 컸습니다.

3. **잊지 못할 경기들과 극적인 승리**  
   - 16강전에서 이탈리아를 2-1로 꺾고,  
   - 8강전에서 스페인을 연장 승부끝에 5-3으로 이겨 승리하는 등, 당시 여러 강팀들을 연파하며 축구 팬들과 미디어의 주목을 받았습니다.  
   이 과정에서 여러 논란과 판정 논란도 있었지만, 한국 축구의 저력과 열정을 보여준 경기라는 평가를 받았습니다.

4. **대표 선수들의 활약**  
   안정환, 홍명보, 박지성, 이영표, 최용수 등 뛰어난 선수들이 팀을 이끌었고, 특히 안정환의 ‘돌려차기 골’과 같은 명장면들이 기억에 남습니다.

5. **국민적 열기와 축구 문화 발전**  
   월드컵 기간 동안 전국에서 축구 열기가 폭발했으며, 축구에 대한 국민적 관심이 크게 높아졌습니다. 이후 한국 축구의 발전과 프로축구 리그(K리그) 성장에도 큰 영향을 끼쳤습니다.

이와 같은 이유들로 2002년 월드컵에서 대한민국은 단순한 참가국 이상의 의미를 지니며 세계 축구사에 한 획을 그은 국가로 기록되었습니다.


- Tools : 앞에서 했었던 function calling 
    - https://docs.langchain.com/oss/python/langchain/tools

In [14]:
# 앞에서 했었던 function calling 
import json
import requests


@tool
def get_weather(location,timezone):
    # 아래의 string은 필수임
    """지역명을 이용해서 위도, 경도를 찾은 후 해당 날씨를 가져오기"""
    # 날씨 api 호출 : 위도 경도를 알아야만 되는 api임
    params = {"name":location,"count":1,"language":"ko"}
    geo = requests.get("https://geocoding-api.open-meteo.com/v1/search",params=params,timeout=30).json()
    results = geo.get("results")[0]
    if not results:
        return json.dumps({"error":f"{location} 위치를 찾지 못했습니다"})
    lat, lon = results.get('latitude'), results.get('longitude')
    print(f"위,경도 {lat},{lon}")


    url = "https://api.open-meteo.com/v1/forecast"

    weather_params = {
        "latitude":lat,
        "longitude":lon,
        "current_weather":"true",
        "timezone":timezone
    }
    weather_result = requests.get(url,params=weather_params,timeout=30).json()
    current_weather = weather_result.get('current_weather')
    if not current_weather:
        return json.dumps({"error":" 날씨 데이터를 가져오지 못했습니다"})
    # 받은 결과 모델에게 돌려주기 위한 json 구조 설정
    return json.dumps({
        "location":location,
        "latitute":lat,
        "longitude":lon,
        "temperature_c": current_weather.get("temperature"),
        "windspeed_kmh": current_weather.get("windspeed_kmh"),
        "winddirection_deg": current_weather.get("winddirection_deg"),
        "weathercode": current_weather.get("weathercode"),
        "time": current_weather.get("time"),

    })

tools = [get_weather]

agent = create_agent(model = model, tools=tools)
res = agent.invoke({"messages":[{"role":"user","content":"Seoul 날씨 어때?"}]})
res['messages'][-1].pretty_print()

위,경도 37.566,126.9784
================================== Ai Message ==================================

서울의 현재 기온은 12.1도이며, 날씨는 맑음(Weather code 0)입니다. 더 궁금한 점 있으시면 말씀해 주세요.


### LangChain 을 사용한 RAG
#### [실습 1] blog 를 기반으로 질의 검색

In [17]:
# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    header_template={
        "User-Agent": "my-langchain-rag-app"
    },
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()
# print(docs) 문서의 양이 크기에 분할 필요
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# 전체 문서를 위의 규칙에 맞추어서 자름
all_splits = text_splitter.split_documents(docs)

# 검색
# 1) 임베딩 모델 생성
# 이전처럼 직접적으로 임베딩을 위한 ai를 호출하지 않고 framwork 사용
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
# 2) 문서의 벡터화 /벡터 DB 저장
vector_store = FAISS.from_documents(all_splits,embeddings)

@tool
def retrieve_context(query):
    """사용자 질문을 받아 벡터 검색 수행 / 관련문서 2개 가져온 후 텍스트 형태로 반환"""
    # 사용자의 질문 임베딩
    retrieve_docs= vector_store.similarity_search(query,k=2)
    serialized = "\n\n".join((f"Source : {doc.metadata}\nContent: {doc.page_content}")
                for doc in retrieve_docs)
    return serialized, retrieve_docs

tools = [retrieve_context]
prompt = """
You have access to a tool that retrieves context from a blog post.
Use the tool to help answer user queries.
"""
agent = create_agent(model, tools, system_prompt=prompt)

query = "What is task docomposition?"

for step in agent.stream({"messages":[{"role":"user","content":query}]}, stream_mode="values"):
    step['messages'][-1].pretty_print()






================================ Human Message =================================

What is task docomposition?
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (call_O1ROC3Dmk7wpK0WLQ8E7DUwz)
 Call ID: call_O1ROC3Dmk7wpK0WLQ8E7DUwz
  Args:
    query: task decomposition
================================= Tool Message =================================
Name: retrieve_context

('Source : {\'source\': \'https://lilianweng.github.io/posts/2023-06-23-agent/\'}\nContent: Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.\nAnother quite distinct approach, LLM+P (Liu et al. 2023), involves relying on an external classical planner to do long-horizon planning. This approach utilizes the Planning Domain Definition Language (PDDL) as an int

#### [실습 2] PDF 기반 질의탐색

In [18]:
# 1. 문서 로드드
pdf_path = "./data/Summary of ChatGPTGPT-4 Research.pdf"

if not Path(pdf_path).exists():
    raise FileNotFoundError(f"PDF not fount : {pdf_path}")

loader = PyPDFLoader(pdf_path)
docs = loader.load()

In [19]:
# 2. 텍스트 분할
# add_start_index : 자른 문서의 index 가 필요하다면 선택적으로 넣으면 된다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

122

In [20]:
print(all_splits[0])

page_content='Summary of ChatGPT/GPT-4 Research
and Perspective Towards the Future of Large
Language Models
Yiheng Liu ∗1, Tianle Han ∗1, Siyuan Ma 1, Jiayue Zhang 1,
Yuanyuan Yang1, Jiaming Tian 1, Hao He 1, Antong Li 2, Mengshen
He1, Zhengliang Liu 3, Zihao Wu 3, Dajiang Zhu 4, Xiang Li 5, Ning
Qiang1, Dingang Shen 6,7,8, Tianming Liu 3, and Bao Ge †1
1School of Physics and Information Technology, Shaanxi Normal University, Xi’an
710119 China
2School of Life and Technology Biomedical-Engineering, Xi’an Jiaotong University,
Xi’an 710049, China
3School of Computing, The University of Georgia, Athens 30602, USA
4Department of Computer Science and Engineering, The University of Texas at
Arlington, Arlington 76019, USA
5Department of Radiology, Massachusetts General Hospital and Harvard Medical
School, Boston 02115, USA
6School of Biomedical Engineering, ShanghaiTech University, Shanghai 201210,
China
7Shanghai United Imaging Intelligence Co., Ltd., Shanghai 200230, China' metadata={'prod

In [21]:
# 3. 벡터 스토어 생성

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = FAISS.from_documents(all_splits,embeddings)

In [22]:
# 4. 검색 tool 작성
@tool
def retrieve_context(query):
    """사용자 질문을 받아 벡터 검색 수행 / 관련문서 k개 가져온 후 텍스트 형태로 반환"""
    # 사용자의 질문 임베딩
    retrieve_docs= vector_store.similarity_search(query,k=4)
    serialized = "\n\n".join((f"Source : {doc.metadata}\nContent: {doc.page_content}")
                for doc in retrieve_docs)
    return serialized, retrieve_docs

tools = [retrieve_context]


In [26]:
# 5. 모델 생성

prompt = """
You have access to a tool that retrieves context from a PDF document.
Use the tool when you need avidence from the PDF.
"""
agent = create_agent(model, tools, system_prompt=prompt)


In [27]:
# 6. 질문 던지기
query = "What can i use chatGPT?"

for step in agent.stream({"messages":[{"role":"user","content":query}]}, stream_mode="values"):
    step['messages'][-1].pretty_print()


================================ Human Message =================================

What can i use chatGPT?
================================== Ai Message ==================================

You can use ChatGPT for a wide range of purposes, including but not limited to:

1. **Answering Questions:** Get explanations, facts, and detailed answers on numerous topics.
2. **Creative Writing:** Help with writing stories, poems, scripts, or brainstorming ideas.
3. **Learning and Education:** Assistance with understanding concepts, summarizing texts, or tutoring in various subjects.
4. **Professional Assistance:** Drafting emails, reports, resumes, and generating professional content.
5. **Coding Help:** Writing, debugging, and explaining code snippets in various programming languages.
6. **Language Translation & Practice:** Translating languages or practicing foreign language conversations.
7. **General Advice:** Offering suggestions on lifestyle, productivity, mental health tips, and more.
8. **

#### [실습 3] 유튜브 영상 자막 추출 후 내용요약

- YoutubeLoader 를 이용한 자막 추출
- RecursiveCharacterTextSplitter 를 이용한 일정한 크기로 자막 분할
- agent 를 이용한 LLM 호출 
- 응답

In [33]:
# 1. 유튜브 자막 로드
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=Pn-W41hC764")
transcript = loader.load()

In [34]:
# 2. 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(transcript)

In [37]:
# -----------------------
# 3. Map 단계 (각 chunk 요약)
# -----------------------
map_prompt= ChatPromptTemplate.from_template("""
Summerize the following transcript chunk:
    ```{text}```
Chunk summery                                            
""")
map_chain = (map_prompt | model | StrOutputParser())

# chunk 요약
chunk_summaries = []
for doc in all_splits:
    summary = map_chain.invoke({"text":doc.page_content})
    chunk_summaries.append(summary)

In [38]:
# -----------------------
# 4. Reduce 단계 (전체 결합 요약)
# -----------------------

combine_prompt = ChatPromptTemplate.from_template("""
You're given summaries of a YouTube's transcript.
Combine them into a final Korean Summary. 
8~10 문장으로 요약하되 핵심문장, 근거, 예시, 결론을 포함해

Summary
```{text}```
Final summary (Korean)                                                                                                                                                                                                                                                        
""")

combine_chain = (combine_prompt | model | StrOutputParser())

final_summary = combine_chain.invoke({"text":"\n\n".join(chunk_summaries)})

print(final_summary)


연설자는 초지능 기계 지능이 고용에 미치는 영향에 대해 기술 혁명이 직업에 큰 변화를 가져오지만 궁극적으로는 더 많고 나은 일자리가 생길 것이라고 전망한다. GPT-4는 자율적 존재가 아닌 도구로서 기존 업무를 보조하며 완전 대체보다는 역할 변화를 가져온다고 설명했다. 기술 발전은 인간의 삶의 질을 높이고 더 도전적이며 만족스러운 일을 추구하도록 만든다고 강조하며, AI와 자동화가 일자리에 미칠 영향에 대응하기 위해 정부 주도의 협력이 필요하다고 밝혔다. IBM의 몽고메리 씨는 AI가 모든 직업을 변화시키고 새로운 직업을 창출할 것이라며, 미래 인력을 AI와 협업할 수 있도록 준비하는 것이 중요하다고 말했다. IBM은 스킬빌드 플랫폼을 통해 7백만 명의 학습자와 1천 개 이상의 강좌를 제공하며 2030년까지 3천만 명을 대상으로 미래 역량 교육을 진행 중이다. AI 모델의 투명성과 과학자의 참여가 필수적이며, 데이터 이해와 평가가 중요하다고 언급했다. AGI가 도달하면 수십 년 내에 노동시장에 깊은 영향을 미칠 수 있으나, 현재 AI는 초기 단계로 향후 발전 가능성이 크다는 점도 함께 강조했다. 마지막으로, 기술 산업이 세상에 끼칠 수 있는 심각한 위험을 인지하고 이를 막기 위한 노력과 협력이 반드시 필요하다는 점을 강조하며 낙관적인 미래를 기대했다.


#### Firecrawl
- 웹사이트를 크롤링하여 LLM(Long Library Management)에서 사용할 수 있는 데이터로 변환
- 접근 가능한 모든 하위 페이지를 크롤링하고 각 페이지에 대한 깔끔한 마크다운과 메타데이터를 제공
- https://docs.langchain.com/oss/python/integrations/document_loaders/firecrawl (api 키 발급 필요)

In [ ]:
# !pip install firecrawl-py
from firecrawl import Firecrawl
app = Firecrawl(api_key="")

# Scrape a website
app.scrape("firecrawl.dev")


[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [42]:
# mode = scrape : 지정된 url 페이지 내용 가져오기 == WebBaseLoader()
# mode = crawl : 하위 페이지 내용 크롤링
loader = FireCrawlLoader(url="firecrawl.dev", mode="scrape")
docs = loader.load()
print(len(docs))

loader = FireCrawlLoader(url="https://naver.com", mode="crawl",params={"limit":5})
docs = loader.load()
print(len(docs))



1
1


#### [실습] react 페이지 크롤링 후 크롤링 문서 기반으로 RAG 적용
- 1. Ingest(색인) 단계: URL들 → Firecrawl → Document → Split → Embedding → FAISS 저장
- 2. Query(질의) 단계: 질문 → FAISS 검색(top-k) → (질문 + 근거) → LLM 답변

In [43]:
import re

loader = FireCrawlLoader(
    url="https://ko.react.dev/reference/react/",
    mode="crawl",
    params={
        "includePaths": [r"^/reference/react/use.*"], 
        "limit": 100,  # 넓게 크롤링
    }
)

docs = loader.load()
print(len(docs))

def get_url(doc):
    md = doc.metadata or {}
    return md.get("source_url") or md.get("source") or md.get("url") or "(unknown)"


# use~ 시작하는 url 만 걸러내기
pattern = re.compile(r"^https://ko\.react\.dev/reference/react/use.*")
use_docs = [d for d in docs if pattern.match(get_url(d))]

print(len(use_docs))

48
19


In [45]:
# 벡터 작업
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = FAISS.from_documents(all_splits,embeddings)

@tool(response_format="content_and_artifact")
def retrieve_context(query):
    """URL 로 수집된 정보를 기반으로 사용자 질문을 받아 벡터 검색 수행 /
      관련문서 k개 가져온 후 텍스트 형태로 반환"""
    # 사용자의 질문 임베딩
    retrieve_docs= vector_store.similarity_search(query,k=4)
    serialized = "\n\n".join((f"Source : {get_url(doc)}\nContent: {doc.page_content}")
                for doc in retrieve_docs)
    return serialized, retrieve_docs

tools = [retrieve_context]


prompt = """
넌 React 공식 문서를 기반으로 답변하는 도우미야.
공식문서 안에서만 대답해. 모르면 모른다고 말해.
답변은 한국어로, 핵심을 불릿으로 정리하고 마지막에 출처 URL  나열해.
"""
agent = create_agent(model, tools, system_prompt=prompt)

query = "UseCallback 에 대해서 설명해"

for step in agent.stream({"messages":[{"role":"user","content":query}]}, stream_mode="values"):
    step['messages'][-1].pretty_print()

================================ Human Message =================================

UseCallback 에 대해서 설명해
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (call_vkreKzRZkwqWOojrV6UmMgir)
 Call ID: call_vkreKzRZkwqWOojrV6UmMgir
  Args:
    query: UseCallback 설명
================================= Tool Message =================================
Name: retrieve_context

Source : https://ko.react.dev/reference/react/useCallback
Content: #### 주의 사항 [Link for 주의 사항 ](https://ko.react.dev/reference/react/useCallback\#caveats "Link for 주의 사항 ")

- `useCallback`은 Hook이므로, **컴포넌트의 최상위 레벨** 또는 커스텀 Hook에서만 호출할 수 있습니다. 반복문이나 조건문 내에서 호출할 수 없습니다. 이 작업이 필요하다면 새로운 컴포넌트로 분리해서 state를 새 컴포넌트로 옮기세요.
- React는 **특별한 이유가 없는 한 캐시 된 함수를 삭제하지 않습니다.** 예를 들어 개발 환경에서는 컴포넌트 파일을 편집할 때 React가 캐시를 삭제합니다. 개발 환경과 프로덕션 환경 모두에서, 초기 마운트 중에 컴포넌트가 일시 중단되면 React는 캐시를 삭제합니다. 앞으로 React는 캐시 삭제를 활용하는 더 많은 기능을 추가할 수 있습니다. 예를 들어, React에 가상화된 목록에 대한 빌트인 지원이 추가한다면, 가상화된 테이블 뷰포트에